***
**Author:** Peter Lu

Data Science Ethics (UCR - Winter 2024)
***

# Regularization and ROC Curves
This notebook will explore a method for helping predictive models better generalize to unseen data. Then, we'll see some absolute metrics that build off of FPR and TNR. They provide interpretable measures for understanding how well a model is understanding the classes in our data.

***
## Regularization
**Regularization** is a technique that attempts to accomplish two goals:
1.  Improve a model's generalizability on unseen data
2.  Keep a model from learning the noise in our data rather than the patterns in the data

How does regularization accomplish this? It penalizes large parameters that overly focus on certain features. Consider a scenario where you learn the following linear regression model to predict continuous labels:
$$y = 1000x_1 + 50x_2 + x_3 + 5$$
where
*  $y$ = House Price
*  $x_1$ = Square Footage
*  $x_2$ = # of rooms
*  $x_3$ = # of shopping centers in a 5 mile radius

Based on the data the model was fit, it learned that Square Footage plays the greatest role, by far, in predicting house price. We may know that other features are more important, but the training data the model learned from didn't represent this fact. Regularization trains a model then "punishes" large coefficients to give smaller coefficients more power in the prediction process. With regularization, we may get a model like this:
$$y_{\text{reg}} = 600x_1 + 500x_2 + 80x_3 + 15.$$

This model will generalize better to unseen data because there's less of a reliance on square footage, a feature that may not be as important in pricing houses in some areas.

Let's see how regularization improves model performance with the MNIST dataset! In this setting, we will train a logistic regression model to classify if an image is a handwritten image of a 5 or not. We'll work with a low number of training samples, with respect to the number of features, to simulate a setting where we have few samples to learn from and many samples to classify. Will regularization help us generalize?

In [1]:
# Import relevant libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.datasets.mnist import load_data

In [ ]:
# Load data + check dimensions
(X_train, y_train), (X_test, y_test) = load_data()
X_train, y_train = X_train[:500], y_train[:500]
print(f'Image Resolution: {X_train.shape[1:]}')
print(f'Number of Features: {X_train.shape[1] * X_train.shape[2]}')
print(f'Training Samples: {X_train.shape[0]}')
print(f'Testing Samples: {X_test.shape[0]}')

In [ ]:
# One-hot encode images as "1" for "image of 5" and "0" for "not image of 5"
y_train_5 = np.where(y_train == 5, 1, 0)
y_test_5 = np.where(y_test == 5, 1, 0)

In [ ]:
# Visualize images of 5
plt.figure(figsize = (8, 6))
for i in range(4):
  plt.subplot(2, 2, i + 1)
  plt.imshow(X_train[y_train_5 == 1][i], cmap = 'gray')
  plt.axis('off')

plt.suptitle('Training Samples: 5')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize images of numbers that aren't 5
plt.figure(figsize = (8, 6))
for i in range(4):
  plt.subplot(2, 2, i + 1)
  plt.imshow(X_train[y_train_5 == 0][i], cmap = 'gray')
  plt.axis('off')

plt.suptitle('Training Samples: Not 5')
plt.tight_layout()
plt.show()

Logistic regression cannot take image matrices as input. They must be flattened, or every row of the image matrix must be concatenated into a long vector! For reference, logistic regression takes the following form:
$$p(y = 1|x) = \frac{1}{1 + e^{-\boldsymbol{\theta}^T\boldsymbol{x}}}$$
$$p(y = 0|x) = 1 - p(y = 1|x) = \frac{e^{-\boldsymbol{\theta}^T\boldsymbol{x}}}{1 + e^{-\boldsymbol{\theta}^T\boldsymbol{x}}}$$

where

$$\boldsymbol{\theta}^T\boldsymbol{x} = \theta_0 + \theta_1x_1 + \theta_2x_2 + \cdots + \theta_nx_n.$$

Notice this is similar to linear regression, but the model is plugged into the funcion above: the **sigmoid function**.


In [ ]:
# Flatten the matrices
X_train_flattened = X_train.reshape(len(X_train), -1)
X_test_flattened = X_test.reshape(len(X_test), -1)

X_train_flattened[0]

In [ ]:
X_train_flattened[0].shape

Scaling or **standardizing** is a common method used to preprocess data for training. It helps models not only learn better because data will be on a similar scale, but they also learn faster. It performs the following transformation across each feature:
$$z = \frac{x - \mu}{\sigma}$$

In [ ]:
# Standardize features for better training
scaler = StandardScaler()
X_train_flattened = scaler.fit_transform(X_train_flattened)
X_test_flattened = scaler.transform(X_test_flattened)

Now, let's compare a regularized model vs. a non-regularized model! There are many types of regularizers, but we'll use L2 regularization with a $C = 0.1$. The lower $C$ is, the stronger the penalization of large coefficients is.

In [ ]:
model_no_reg = LogisticRegression(penalty = None)
model_no_reg.fit(X_train_flattened, y_train_5)
print(f'Training Accuracy: {model_no_reg.score(X_train_flattened, y_train_5):.2%}')
print(f'Test Accuracy: {model_no_reg.score(X_test_flattened, y_test_5):.2%}')

In [ ]:
model_reg = LogisticRegression(penalty = 'l2', C = 0.1)
model_reg.fit(X_train_flattened, y_train_5)
print(f'Training Accuracy: {model_reg.score(X_train_flattened, y_train_5):.2%}')
print(f'Test Accuracy: {model_reg.score(X_test_flattened, y_test_5):.2%}')

***
### ROC Curves
The **receiver operating characteristic (ROC) curve** plots the *true positive rate (TPR)* vs. the *false positive rate (FPR)* against each other at varying thresholds.

Given a model, we use its predicted probabilities that a sample is a given class (0/1) and provide a threshold for how to determine if a sample falls into the class 0 or class 1. For example, if a model predicts sample $x$ to have a predicted probability of 0.6 of being in class 1, a threshold value of 0.8 would now label $x$ as being in class 0 because it did not exceed 0.8.

We do this procedure for several different threshold values from 0 to 1. This gives us different TPRs and FPRs that we can plot. Let's see this procedure below.

In [ ]:
# Compute probabilities of being a 5
y_proba_no_reg = model_no_reg.predict_proba(X_test_flattened)[:, 1]
y_proba_reg = model_reg.predict_proba(X_test_flattened)[:, 1]

In [ ]:
# Plot non-5
plt.imshow(X_test[0], cmap = 'gray')
plt.title(f'Probability of 5 with no reg: {y_proba_no_reg[0]:.2}')
plt.axis('off')
plt.show()

In [ ]:
# Plot 5
img_5 = y_test_5 == 1
plt.imshow(X_test[img_5][10], cmap = 'gray')
plt.title(f'Probability of 5 with no reg: {y_proba_no_reg[img_5][10]:.2}')
plt.axis('off')
plt.show()

In [ ]:
# Compute ROC curves for regularized and non-regularized models
from sklearn.metrics import roc_curve, auc
fpr_no_reg, tpr_no_reg, thresholds_no_reg = roc_curve(y_test_5, y_proba_no_reg)
fpr_reg, tpr_reg, thresholds_reg = roc_curve(y_test_5, y_proba_reg)

In [ ]:
# Example of how ROC computes an FPR using a threshold
from sklearn.metrics import confusion_matrix
e = thresholds_no_reg[500]
y_new = np.where(y_proba_no_reg > e, 1, 0)
tn, fp, fn, tp = confusion_matrix(y_test_5, y_new).ravel()
print(f'Computed FPR: {fp / (fp + tn)}', end = '\n\n')

print(f'Threshold: {e}')
print(f'Threshold FPR: {fpr_no_reg[500]}')

***
### Area Under the Curve (AUC)
After computing the FPR and TPR for several different thresholds, we then compute the **area under the curve (AUC)** which is a measure of a model's performance in distinguishing between two classes. Generally,
*  AUC = 1 means a model perfectly classifies the two classes
*  AUC = 0.5 means a model classifies equivalently to a coin flip

Lower the AUC, the worse the model is at distinguishing the two classes.

In [ ]:
auc_no_reg = auc(fpr_no_reg, tpr_no_reg)
auc_reg = auc(fpr_reg, tpr_reg)

plt.plot(fpr_no_reg, tpr_no_reg, color = 'red', label = f'No Reg AUC: {auc_no_reg:.3f}')
plt.plot(fpr_reg, tpr_reg, color = 'green', label = f'Reg AUC: {auc_reg:.3f}')
plt.title('ROC Curves: Reg vs. Non-Reg')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()
plt.legend()
plt.show()

In [ ]:
Cs = np.linspace(0.00001, 1, 10)
for C in Cs:
  model = LogisticRegression(penalty = 'l2', C = C)
  model.fit(X_train_flattened, y_train_5)
  y_proba = model.predict_proba(X_test_flattened)[:, 1]
  fpr, tpr, _ = roc_curve(y_test_5, y_proba)
  area = auc(fpr, tpr)
  plt.plot(fpr, tpr, label = f'AUC: {area:.3f}, C = {C:.2f}')

plt.title('ROC Curves with varying C\'s')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.show()

***
Now that we've seen how to determine a model's capability at distinguishing between two classes, what can ROC curves and AUC tell us about bias and fairness? Let's load the COMPAS dataset and see if the bias/unfairness we detected before is a result of our model or a systemic issue.

In [ ]:
# Load and preprocess data as in assignment 1

url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
df_compas = pd.read_csv(url)

# Grab useful columns
cols_to_keep = ["id", "age", "c_charge_degree", "race", "age_cat", "score_text",
                "sex", "priors_count", "days_b_screening_arrest",
                "decile_score", "is_recid", "two_year_recid"]

df_selected = df_compas[cols_to_keep].copy()


# Grab samples of interest
df_analysis = df_selected[
    (df_selected.score_text != "N/A") &
    (df_selected.days_b_screening_arrest <= 30) &
    (df_selected.days_b_screening_arrest >= -30) &
    (df_selected.is_recid != -1) &
    (df_selected.c_charge_degree != "O")
    ].copy()

df_analysis["decile_score"] = pd.to_numeric(df_analysis["decile_score"])
df_logistic = df_analysis.copy()

# one-hot encoding
df_logistic = pd.get_dummies(df_logistic,
                             columns = ["c_charge_degree", "race",
                                        "age_cat", "sex"])

# Replace 'Low' with 0 and `Medium` / `High` with 1
df_logistic["label"] = np.where(df_logistic["score_text"] != "Low", 1, 0)
df_logistic.drop(columns = ['score_text'], inplace = True)

# Format column names
df_logistic.columns = df_logistic.columns.str.replace(' ', '_')
df_logistic.columns = df_logistic.columns.str.replace('-', '_')

renamed_cols = {'age_cat_25___45':'age_cat_25_to_45',
                'c_charge_degree_F':'Felony',
                'c_charge_degree_M':'Misdemeanor'}

df_logistic = df_logistic.rename(columns = renamed_cols)
X = df_logistic[['priors_count', 'two_year_recid', 'Misdemeanor', 'age_cat_Greater_than_45', 'age_cat_Less_than_25', 'race_African_American', 'race_Asian', 'race_Hispanic', 'race_Native_American', 'race_Other', 'sex_Female']]
y = df_logistic['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.95, random_state = 5)
print(f'Training samples: {X_train.shape[0]}')
print(f'Test samples: {X_test.shape[0]}')

In [ ]:
# Train the models and check their accuracies
model_no_reg = LogisticRegression(penalty = None)
model_no_reg.fit(X_train, y_train)

print(f'Training Accuracy: {model_no_reg.score(X_train, y_train):.2%}')
print(f'Test Accuracy: {model_no_reg.score(X_test, y_test):.2%}')

In [ ]:
model_reg = LogisticRegression(penalty = 'l2', C = 0.1)
model_reg.fit(X_train, y_train)

print(f'Training Accuracy: {model_reg.score(X_train, y_train):.2%}')
print(f'Test Accuracy: {model_reg.score(X_test, y_test):.2%}')

In [ ]:
# Generate ROC curves and calculate AUC
y_proba_no_reg = model_no_reg.predict_proba(X_test)[:, 1]
y_proba_reg = model_reg.predict_proba(X_test)[:, 1]

fpr_no_reg, tpr_no_reg, thresholds_no_reg = roc_curve(y_test, y_proba_no_reg)
auc_no_reg = auc(fpr_no_reg, tpr_no_reg)

fpr_reg, tpr_reg, thresholds_reg = roc_curve(y_test, y_proba_reg)
auc_reg = auc(fpr_reg, tpr_reg)

In [ ]:
plt.plot(fpr_no_reg, tpr_no_reg, color = 'red', label = f'No Reg AUC: {auc_no_reg:.3f}')
plt.plot(fpr_reg, tpr_reg, color = 'green', label = f'Reg AUC: {auc_reg:.3f}')
plt.title('ROC Curves: Reg vs. Non-Reg')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()
plt.legend()
plt.show()

From the above plot, this tells us that even after regularizing, the bias/unfairness we saw before is *most likely* not a result of our model. We can make this conclusion because regularization penalizes features that may overly influence a model's decision. After regularization, our predictive power remained about the same, meaning that there's no feature or set of features overly influencing our predictions. In other words, the model isn't biased toward race, sex, or age group. This *may suggest* the bias/unfairness is **systemic**.

Let's check this hypothesis specifically on the models' performance on samples from the African American population.

In [ ]:
# Grab samples from African American community
test_idxs = np.where(X_test['race_African_American'] == 1)
aa_proba_no_reg = model_no_reg.predict_proba(X_test.iloc[test_idxs])[:, 1]
aa_proba_reg = model_reg.predict_proba(X_test.iloc[test_idxs])[:, 1]

In [ ]:
fpr_no_reg, tpr_no_reg, thresholds_no_reg = roc_curve(y_test.iloc[test_idxs], aa_proba_no_reg)
auc_no_reg = auc(fpr_no_reg, tpr_no_reg)

fpr_reg, tpr_reg, thresholds_reg = roc_curve(y_test.iloc[test_idxs], aa_proba_reg)
auc_reg = auc(fpr_reg, tpr_reg)

In [ ]:
plt.plot(fpr_no_reg, tpr_no_reg, color = 'red', label = f'No Reg AUC: {auc_no_reg:.3f}')
plt.plot(fpr_reg, tpr_reg, color = 'green', label = f'Reg AUC: {auc_reg:.3f}')
plt.title('ROC Curves: Reg vs. Non-Reg')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()
plt.legend()
plt.show()

In [ ]:
Cs = np.linspace(0.00001, 1, 10)
X_test_aa = X_test.iloc[test_idxs]
y_test_aa = y_test.iloc[test_idxs]

for i, C in enumerate(Cs):
  if i == 0:
    continue
  model = LogisticRegression(penalty = 'l2', C = C)
  model.fit(X_train, y_train)
  y_proba = model.predict_proba(X_test_aa)[:, 1]
  fpr, tpr, _ = roc_curve(y_test_aa, y_proba)
  area = auc(fpr, tpr)
  plt.plot(fpr, tpr, label = f'AUC: {area:.3f}, C = {C:.2f}')

plt.title('ROC Curves with varying C\'s')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.show()

### What now?
To further investigate:
*  Investigate how the data was collected
*  Try different models with different levels of regularization
*  Test more complex models that may capture the bias/unfairness we saw previously